
### Pipeline de Preprocesamiento ### 
- hashing, pseudoanonimización, winsorizar, estandarizar variables numéricas


In [64]:
# Importación de librerías

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import hashlib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
## TRATAMIENTO DE INFORMACIÓN SENSIBLE
# Transformador personalizado para hashing de emails
# Función para hashear el email
def hash_email(email):
    email = email.lower()  # Convertir a minúsculas
    sha256 = hashlib.sha256()  # Crear el objeto hash
    sha256.update(email.encode('utf-8'))  # Codificar el email a bytes y aplicar hashing
    return sha256.hexdigest()  # Retornar el hash en formato hexadecimal

# Transformador personalizado para hashing de emails
class EmailHasher(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # No necesita ajuste

    def transform(self, X):
        X['Email'] = X['Email'].apply(hash_email)  # Aplicar hashing a la columna 'Email'
        return X  # Retornar el DataFrame transformado


In [66]:
# Transformador personalizado para Pseodoanonimizar Direcciones

class AddressPseudoanonymizer(BaseEstimator, TransformerMixin):
    def __init__(self, address_column='Address'):
        self.address_column = address_column
        self.address_to_code = {}  # Diccionario para almacenar códigos pseudoanónimos

    def fit(self, X, y=None):
        # Crear un diccionario que asigna códigos únicos a cada dirección
        unique_addresses = X[self.address_column].unique()
        self.address_to_code = {address: f'ADDR{index + 1}' for index, address in enumerate(unique_addresses)}
        return self  # Retornar self para mantener la estructura de scikit-learn

    def transform(self, X):
        # Reemplazar las direcciones en el DataFrame por sus códigos correspondientes
        X[self.address_column] = X[self.address_column].map(self.address_to_code)
        return X  # Retornar el DataFrame transformado


In [67]:
# Transformador personalizado para Winsorizar

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns  # Columnas que serán winsorizadas
        self.limits_ = {}  # Aquí guardaremos los límites IQR calculados

    # Método para calcular límites basados en IQR
    def calculate_iqr_limits(self, df, columns):
        limits_dict = {}
        for column in columns:
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            limits_dict[column] = (lower_bound, upper_bound)

        return limits_dict

    def fit(self, X, y=None):
        # Calcular los límites de winsorización basados en el IQR para cada columna
        self.limits_ = self.calculate_iqr_limits(X, self.columns)
        return self

    def transform(self, X):
        X_copy = X.copy()  # Crear una copia del DataFrame para no modificar el original
        # Aplicar winsorización utilizando los límites calculados
        for column in self.columns:
            lower_bound, upper_bound = self.limits_[column]
            X_copy[column] = np.clip(X_copy[column], lower_bound, upper_bound)
        return X_copy



In [68]:
# Transformador Personalizado para Estandarización

class NumericalStandardizer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        # Ajustar el escalador solo a las columnas especificadas
        self.scaler.fit(X[self.columns])
        return self

    def transform(self, X):
        # Aplicar la estandarización
        X[self.columns] = self.scaler.transform(X[self.columns])
        return X


In [69]:
columns_to_check = ['Avg. Session Length', 'Time on App', 'Time on Website', 'Length of Membership']

In [70]:
columns_to_standardize = ['Length of Membership', 'Time on App', 'Avg. Session Length']

In [77]:
# Crear el pipeline
pipeline = Pipeline([
    ('email_hasher', EmailHasher()),
    ('address_pseudoanonymizer', AddressPseudoanonymizer(address_column='Address')),
    ('winsorizer', Winsorizer(columns=columns_to_check)),
    ('standardizer', NumericalStandardizer(columns=columns_to_standardize)) 
])

In [72]:
## Cargar el dataset desde el repositorio de GITHUB
url = 'https://raw.githubusercontent.com/ITACADEMYprojectes/projecteML/main/Ecommerce_Customers.csv'
df = pd.read_csv(url)

In [73]:
train, test = train_test_split(df, test_size=0.2, random_state=22)

In [ ]:
# Aplicar el pipeline al conjunto de entrenamiento
train_transformed = pipeline.fit_transform(train)

# Aplicar el pipeline al conjunto de prueba
test_transformed = pipeline.transform(test)

# Verificar que las transformaciones se han aplicado
print(train_transformed.head())
print(test_transformed.head())
